In [1]:
#!pip install transformers

In [2]:
#!pip install sentence-transformers

In [1]:
#!pip install transformers
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import re
import sys
import glob
import random
import torch.nn as nn
import xml.etree.ElementTree as ET
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import random
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
import ast
import scipy
# from argparse import ArgumentParser
# parser = ArgumentParser()
# parser.add_argument("-multi_lb_classi", default=1, type=int, help="whether train a multi-lable classifier or individual classifiers for each class")
# opt = parser.parse_args()
multi_lb_classi = 0


def preprocess(example_sent):
    global stop_words
    word_tokens = word_tokenize(example_sent.lower())
    filtered_sentence = [w for w in word_tokens if not w in stop_words and w.isalpha()]
    new = " " 
    a = new.join(filtered_sentence)
    return a


2021-12-08 10:29:36.346336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/apps/cuDNN/7.6.5-cuda-10.2/lib64
2021-12-08 10:29:36.346397: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_dataset_2018(files, folder):
    
    pairs_lis = []
    prev_corpus = []
    docs_wise = {}
    
    for z,f in enumerate(files):
        cit_text_lis = []
        ref_text_lis = []
        cit_off_lis = []
        ref_off_lis = []
        new_corpus = []

        try:
            a = folder+f+"/Reference_XML/"+f+".xml"
            tree = ET.parse(a)
            root = tree.getroot()
            final =[]
            total = len(root)
            for a in root:
                for b in a:
                    final.append(b.text)
            d={'col1':final}
            rp = pd.DataFrame(data=d)
            corpus = rp.col1
            new_corpus = corpus.apply(lambda x: preprocess(x))

            data = None
            ann = None
            a_folder = folder+f+"/annotation/"
            file = os.listdir(a_folder)[0]

            ann = a_folder+file
            with open(ann,"r") as file:
                data = file.read()

                cit_text = re.findall("Citation Text:\s+([^|]*)", data)
                pattern = r'\<.*?\>'
                pattern2 = r'\(.*?\)'
                for c in cit_text:
                    c = re.sub(pattern2,'',re.sub(pattern, '', c))
                    c = preprocess(c)
                    cit_text_lis.append(c)


                ref_text = re.findall("Reference Text:\s+([^|]*)", data)
                pattern = r'\<.*?\>'
                pattern2 = r'\(.*?\)'
                for ref in ref_text:
                    ref = re.sub(pattern2,'',re.sub(pattern, '', ref))
                    ref = preprocess(ref)
                    ref_text_lis.append(ref)
                    

    #                 cit_off = re.findall("Citation Offset:\s+([^|]*)", data)
    #                 for c in cit_off:
    #                     c = ast.literal_eval(c)
    #                     cit_off_lis.append(c)


                ref_off = re.findall("Reference Offset:\s+([^|]*)", data)
                for r in ref_off:
                    r = ast.literal_eval(r)
                    ref_off_lis.append(r)

            for i in range(len(cit_text_lis)):
                for j in ref_off_lis[i]:
                    if int(j) < len(new_corpus):
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[int(j)]],label=1.0)) #positive pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3)) #negative pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
                        pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
                        pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))

            if (z!=0 and len(new_corpus)!=0 and len(prev_corpus)!=0):
                pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
                pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
                pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
                pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
                
            docs_wise[f] = {'corpus':new_corpus.values,  'cite_text':cit_text_lis, 'ref_off':ref_off_lis}
            prev_corpus = new_corpus
        except Exception as e: 
            print(f,e)
#             exc_type, exc_obj, exc_tb = sys.exc_info()
#             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#             print(exc_type, fname, exc_tb.tb_lineno)
#             prev_corpus = []
        
        
            
        
        
        
        
            
            
    return pairs_lis, docs_wise


In [12]:
docs = os.listdir("./scisumm-2018/Training")
# random.shuffle(docs)
train_e = int(0.7*len(docs))
val_e = int(0.1*len(docs)) + train_e
train_rps = docs[:train_e]
val_rps = docs[train_e:val_e]
test_rps = docs[val_e:]

In [17]:
# test_docs  = os.listdir("./scisumm-2018/Test")
stop_words = set(stopwords.words('english'))
train_data, train_docs = get_dataset_2018(train_rps,"scisumm-2018/Training/")
val_data, val_docs = get_dataset_2018(val_rps,"scisumm-2018/Training/")
test_data, test_docs = get_dataset_2018(test_rps,"scisumm-2018/Training/")


J98-2005 not well-formed (invalid token): line 36, column 116
P98-1081 not well-formed (invalid token): line 147, column 30
E03-1020 not well-formed (invalid token): line 34, column 62
H89-2014 not well-formed (invalid token): line 82, column 106
X96-1048 not well-formed (invalid token): line 14, column 117
N01-1011 not well-formed (invalid token): line 13, column 100
J00-3003 not well-formed (invalid token): line 12, column 25
H05-1115 not well-formed (invalid token): line 6, column 109
C94-2154 not well-formed (invalid token): line 10, column 202
N09-1001 invalid syntax (<unknown>, line 1)
W06-3909 invalid syntax (<unknown>, line 1)


In [19]:
def test_if_valid_data(docs):
    for k in docs:
        try:
            assert(len(docs[k]['ref_off']) == len(docs[k]['cite_text']))
        except:
            print("test failed: length of queries and gt not equal")
            return
    print("unit test passed")
test_if_valid_data(train_docs)
test_if_valid_data(val_docs)
test_if_valid_data(test_docs)

unit test passed
unit test passed
unit test passed


In [20]:
len(train_data), len(val_data),len(test_data), len(train_docs), len(val_docs),len(test_docs)

(3912, 746, 1056, 22, 3, 4)

In [21]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_data, shuffle=True, batch_size=1)
test_dataloader = DataLoader(test_data, shuffle=True, batch_size=1)
                

In [22]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')
train_loss = losses.CosineSimilarityLoss(model)
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

Iteration:   0%|          | 0/245 [00:00<?, ?it/s]

In [93]:
import torch
torch.save(model,'bert2018_fine_tuned.pth')

In [94]:
model = torch.load('bert2018_fine_tuned.pth')

In [23]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [24]:
# from ignite.metrics import Rouge  #https://pypi.org/project/pytorch-ignite/
from rouge_score import rouge_scorer  #https://pypi.org/project/rouge-score/
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
from pythonrouge.pythonrouge import Pythonrouge


def get_matching_sentences(model,corpus,queries,gt, thresh=0.6,topk=3,cs_based=True):
    #f1 score will be same as precision and recall in our case: since documents
    
    # Get a vector for each headline (sentence) in the corpus
    corpus_embeddings = model.encode(corpus)
    corpus_embeddings = corpus_embeddings/np.linalg.norm(corpus_embeddings,axis=0).reshape(-1)
    # Define search queries and embed them to vectors as well

    query_embeddings = model.encode(queries)
    # For each search term return 3 closest sentences
    total_nums_correct = 0
    total_retrieved = 0
    total_relevent = 0
    rouge1 = []
    rouge2 = []
    rouge_su4 = []
    for i in range(len(queries)):
        query, query_embedding  = queries[i], query_embeddings[i]
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
        
        distances = np.array(distances)
        distances = 1 - distances

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: -x[1])
#         print(results)
        retrieved = []
        if cs_based:
            for k,dist in results:
                if dist >= thresh:
                    retrieved.append(k)
        else: #retrieve topk most matching sentences
            indexes = results[0:topk]
            retrieved = []
            for l,k in indexes:
                retrieved.append(l)
    
        nums_correct = len(np.intersect1d(retrieved,gt[i]))
        total_nums_correct += nums_correct
        total_relevent += len(gt[i])
        total_retrieved += len(retrieved)
        for idx in retrieved:
            scores = scorer.score(corpus[idx].strip(), query)
            rouge = Pythonrouge(summary_file_exist=False,
                    summary=[[corpus[idx].strip()]], reference=[[[query]]],
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                    recall_only=True, stemming=True, stopwords=True,
                    word_level=True, length_limit=True, length=50,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=1000, favor=True, p=0.5)
    
            score = rouge.calc_score()
            rouge1.append(score['ROUGE-1'])
            rouge2.append(score['ROUGE-2'])
            rouge_su4.append(score['ROUGE-SU4'])

            
    return total_nums_correct, total_relevent,total_retrieved, np.mean(rouge1), np.mean(rouge2), np.mean(rouge_su4)
    
                    
#             print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

In [25]:
docs_keys_train = list(train_docs.keys())
docs_keys_train

['P06-2124',
 'E09-2008',
 'D10-1083',
 'W04-0213',
 'C10-1045',
 'C90-2039',
 'P00-1025',
 'D09-1023',
 'C98-1097',
 'W95-0104',
 'N04-1038',
 'N09-1025',
 'P07-1040',
 'W11-0815',
 'D10-1058',
 'C02-1025',
 'W03-0410',
 'W09-0621',
 'P98-1046',
 'P05-1053',
 'N06-2049',
 'W08-2222']

In [26]:
test_docs.keys()

dict_keys(['C08-1098', 'P05-1004', 'J96-3004', 'C00-2123'])

In [27]:
for k in test_docs:
    print(len(test_docs[k]['ref_off']), len(test_docs[k]['cite_text']))

28 28
13 13
64 64
18 18


In [36]:
os.environ['TOKENIZERS_PARALLELISM']= '0'
def evaluate(model, docs, thresh=0.6,topk=3,cs_based=True):
    rouge1_lis = []
    rouge2_lis = []
    rouge_su4_lis = []
    tp_big = 0
    tot_relevent = 0
    tot_retrieved = 0

    for k in docs:
        tp, rele,ret, rouge1, rouge2, rouge_su4 = get_matching_sentences(model,docs[k]['corpus'], docs[k]['cite_text'], docs[k]['ref_off'],thresh,topk,cs_based)
        tp_big += tp
        tot_relevent += rele
        tot_retrieved += ret
        rouge1_lis.append(rouge1)
        rouge2_lis.append(rouge2)
        rouge_su4_lis.append(rouge_su4)
    recall = tp_big/tot_relevent
    precision = tp_big/tot_retrieved
    f1 = 2*recall*precision/(recall+precision+1e-10)
    return recall,precision,f1, np.mean(rouge1_lis), np.mean(rouge2_lis), np.mean(rouge_su4_lis)

In [29]:


recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,train_docs,thresh=0.65)   
print("metrics obtained in train: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

metrics obtained in train: recall 0.3609375, precision 0.1763358778625954, f1-score 0.23692307687897962, rouge1 0.20644336908995095, rouge2 0.05007303001994657, rouge_su4 0.0692603719023074


In [30]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.65)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

metrics obtained in test: recall 0.005747126436781609, precision 0.004424778761061947, f1-score 0.004999999950845001, rouge1 0.20585767107793065, rouge2 0.047522497145947244, rouge_su4 0.05414208794237818


In [33]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.6)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

metrics obtained in test: recall 0.017241379310344827, precision 0.005119453924914676, f1-score 0.007894736806799169, rouge1 0.1754443025850619, rouge2 0.03489439339590088, rouge_su4 0.04993661360795343


In [34]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.55)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

metrics obtained in test: recall 0.040229885057471264, precision 0.004989308624376337, f1-score 0.008877615706429738, rouge1 0.14744871211600644, rouge2 0.0235749694857687, rouge_su4 0.04140865457807666


In [37]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,cs_based=False)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

metrics obtained in test: recall 0.034482758620689655, precision 0.016260162601626018, f1-score 0.02209944747026037, rouge1 0.15032574190132786, rouge2 0.02408980974320818, rouge_su4 0.04102838252314815
